In [1]:
# import SpamEmailBayes class
from spamEmail import SpamEmailBayes
import re

In [2]:
# Create SpamEmailBayes instance
spam = SpamEmailBayes()

In [3]:
# Init dictory to save words and amount
spamDict = {}
normDict = {}
testDict = {}

In [4]:
# Init storege to save words in each email
wordsList = []
wordsDict = {}

In [5]:
# Init test result dictory
# Set file name as the key and category as the value
testResult = {}

In [6]:
# Get the normal, spam and test email file name list
normFileList = spam.get_file_list('../data/normal/')
spamFileList = spam.get_file_list('../data/spam')
testFileList = spam.get_file_list('../data/test')

In [7]:
# Get the number of noramla and spam email
normFileLen = len(normFileList)
spamFileLen = len(spamFileList)

In [8]:
# Get the stop words list
stop_list = spam.get_stop_words()

In [9]:
# Get the word frequence in normal email
for file_name in normFileList:
    wordsList.clear()
    with open('../data/normal/' + file_name, 'r', encoding='gbk') as f:
        content = f.read()
        for line in content:
            rule = re.compile(u'[^\u4E00-\u9FA5]')
            line = rule.sub(r'', line)
            spam.get_word_list(line, wordsList, stop_list)
    
    spam.addToDict(wordsList, wordsDict)

normDict = wordsDict.copy()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jc/l9vx9tp979g0tm976wjrgwkr0000gn/T/jieba.cache
Loading model cost 0.863 seconds.
Prefix dict has been built succesfully.


In [10]:
# Get the word frequence in spam email
for file_name in spamFileList:
    wordsList.clear()
    with open('../data/spam/' + file_name, 'r', encoding='gbk') as f:
        content = f.read()
        for line in content:
            rule = re.compile(u'[^\u4E00-\u9FA5]')
            line = rule.sub(r'', line)
            spam.get_word_list(line, wordsList, stop_list)
    
    spam.addToDict(wordsList, wordsDict)

spamDict = wordsDict.copy()

In [11]:
# Get the word frequence in test email
for file_name in testFileList:
    testDict.clear()
    wordsDict.clear()
    wordsList.clear()
    with open('../data/test/' + file_name, 'r', encoding='gbk') as f:
        content = f.read()
        for line in content:
            rule = re.compile(u'[^\u4E00-\u9FA5]')
            line = rule.sub(r'', line)
            spam.get_word_list(line, wordsList, stop_list)
    
    spam.addToDict(wordsList, wordsDict)
    testDict = wordsDict.copy()
    
    # Get the top 15 words according to the P(s|w) in each test email
    word_prob_list = spam.get_test_words(testDict, spamDict, normDict, normFileLen, spamFileLen)
  
    # Compute the probability by naive bayes method
    p = spam.calBayes(word_prob_list)
    
    # Set category tag for each test email
    if (p > 0.9):
        testResult.setdefault(file_name, 1)
    else:
        testResult.setdefault(file_name, 0)

In [12]:
# Compute the test accuracy
# the file name lower than 1000 are normal email in test files
testAccuracy = spam.calAccuracy(testResult)
# for file, cat in testResult.items():
#     print(file + '/' + str(cat))
print(testAccuracy)

0.5076530612244898
